# Notebook for delivering of the Airgo project

### Imports

In [ ]:
import grid2op
from grid2op.Agent import TopologyGreedy, RecoPowerlineAgent
from grid2op.Runner import Runner
from grid2op.Reward import L2RPNReward

import os
import sys
from pathlib import Path

sys.path.insert(0, '../')

import src

from src.Backend.powsybl_backend import PowsyblBackend

### Paths to the data

In [ ]:
path = Path(os.getcwd())
root_path = path.parent.absolute()
ieee14_path = os.path.join(root_path,os.path.join("src",os.path.join("data_test","l2rpn_case14_sandbox_Pypowsybl")))
ieee1888_path = os.path.join(root_path,os.path.join("src",os.path.join("data_test","case1888rte_Artelys_v1")))


### Evaluation function

In [ ]:
from l2rpn_baselines.utils import save_log_gif 

def evaluate(env,
             grid2op_agent,
             nb_episode=1,
             nb_process=1,
             max_steps=-1,
             verbose=False,
             save_gif=False,
             logs_path=os.path.join(root_path,"logs")):
    
    if nb_episode == 0:
        return grid2op_agent, []

    # Build runner
    runner_params = env.get_params_for_runner()
    runner_params["verbose"] = verbose
    runner = Runner(**runner_params,
                    agentClass=None,
                    agentInstance=grid2op_agent)
    
    # Run the agent on the scenarios
    if logs_path is not None:
        os.makedirs(logs_path, exist_ok=True)

    res = runner.run(path_save=logs_path,
                     nb_episode=nb_episode,
                     nb_process=nb_process,
                     max_iter=max_steps,
                     pbar=verbose,
                     )

    # Print summary
    if verbose:
        print("Evaluation summary:")
        for _, chron_name, cum_reward, nb_time_step, max_ts in res:
            msg_tmp = "chronics at: {}".format(chron_name)
            msg_tmp += "\ttotal score: {:.6f}".format(cum_reward)
            msg_tmp += "\ttime steps: {:.0f}/{:.0f}".format(nb_time_step, max_ts)
            print(msg_tmp)

    if save_gif:
        if verbose:
            print("Saving the gif of the episodes")
        save_log_gif(logs_path, res)

### Loading env and evaluation of an agent

In [ ]:
env = grid2op.make(
#         ieee14_path,
        ieee1888_path,
        reward_class = L2RPNReward,
        backend = PowsyblBackend(detailed_infos_for_cascading_failures=False)
    )

evaluate(
    env,
    RecoPowerlineAgent(env.action_space),
    verbose=True,
    nb_episode=4,
#     save_gif = True,
    max_steps=50,
)